In [50]:
import psycopg2
from imessage_reader import fetch_data
from os.path import expanduser
import sys
import sqlite3
import datetime as dt
import plotly.graph_objects as go
import pandas as pd


In [2]:
def postgre_exe(query):
    cursor.execute(query)
    cursor.execute("COMMIT")

In [7]:
db = psycopg2.connect(host = 'localhost', dbname = 'ledger', user = 'mato', port = 5432)
cursor = db.cursor()

In [14]:
query = """SELECT * FROM LEDGER.TRANSACTION;"""
cursor.execute(query)
results = cursor.fetchall()

In [16]:
results[0]

(1,
 datetime.date(2023, 1, 1),
 datetime.time(10, 11),
 'Sun',
 2,
 1,
 4900,
 '네이버플러스구독요금',
 '네이버파이낸셜',
 2)

In [1]:
query = """
SELECT A.TRANS_ID,
        A.TRANS_DATE,
        A.TRANS_TIME,
        B.CATEGORY_NAME,
        A.AMOUNTS,
        A.PLACE
FROM
        LEDGER.TRANSACTION AS A
        JOIN
            LEDGER.CATEGORY AS B
            ON A.CATEGORY = B.CATEGORY_ID;
"""
cursor.execute(query)
results = cursor.fetchall()
results

NameError: name 'cursor' is not defined

In [18]:
results[0]

(1, datetime.date(2023, 1, 1), datetime.time(10, 11), '식비', 4900, '네이버파이낸셜')

In [19]:
results[0][1]

datetime.date(2023, 1, 1)

In [8]:
today = dt.date.today()
month_first = dt.date(today.year, today.month, 1).strftime('%Y-%m-%d')
query = f"""SELECT TYPE, SUM(AMOUNTS) AS AMOUNT_SUM FROM LEDGER.TRANSACTION WHERE TRANS_DATE > '{month_first}' GROUP BY TYPE;"""
cursor.execute(query)
test = cursor.fetchall()

print(test)

[(2, 652025)]


### draw a graph

In [10]:
db = psycopg2.connect(host = 'localhost', dbname = 'ledger', user = 'mato', port = 5432)
cursor = db.cursor()

In [21]:
query = """SELECT TO_CHAR(TRANS_DATE, 'YYYY-MM-DD'), SUM(AMOUNTS) AS AMOUNTS_SUM FROM LEDGER.TRANSACTION 
        WHERE EXTRACT(MONTH FROM TRANS_DATE) = EXTRACT(MONTH FROM CURRENT_DATE) 
        AND TYPE = 2
        GROUP BY TRANS_DATE;"""
cursor.execute(query)
results = cursor.fetchall()

In [25]:
pd.DataFrame(results, columns = ['DATE', 'AMOUNTS'])

,DATE,AMOUNTS
0,2023-02-01,19400
1,2023-02-02,12700
2,2023-02-03,86950
3,2023-02-04,139600
4,2023-02-05,77600
5,2023-02-06,16100
6,2023-02-07,17600
7,2023-02-08,11900
8,2023-02-09,12700
9,2023-02-10,6700


In [43]:
query = """
SELECT TO_CHAR(TRANS_DATE, 'YYYY-MM-DD'), SUM(AMOUNTS) AS AMOUNTS_SUM FROM LEDGER.TRANSACTION
WHERE TRANS_DATE >= DATE_TRUNC('MONTH', CURRENT_DATE - INTERVAL '1' MONTH)
    AND TRANS_DATE < DATE_TRUNC('MONTH', CURRENT_DATE)
    AND TYPE = 2
    GROUP BY TRANS_DATE
    ORDER BY TRANS_DATE ASC;
"""
cursor.execute(query)
results = cursor.fetchall()

In [70]:
payments_per_daily =  pd.DataFrame(results, columns = ['date', 'amounts'])

In [71]:
payments_per_daily

,date,amounts
0,2023-02-01,19400
1,2023-02-02,32100
2,2023-02-03,119050
3,2023-02-04,258650
4,2023-02-05,336250
5,2023-02-06,352350
6,2023-02-07,369950
7,2023-02-08,381850
8,2023-02-09,394550
9,2023-02-10,401250


In [62]:
query = """
SELECT TO_CHAR(TRANS_DATE, 'YYYY-MM-DD'), SUM(AMOUNTS) AS AMOUNTS_SUM FROM LEDGER.TRANSACTION
WHERE TRANS_DATE >= DATE_TRUNC('MONTH', CURRENT_DATE)
    AND TYPE = 2
    GROUP BY TRANS_DATE
    ORDER BY TRANS_DATE ASC;
"""
cursor.execute(query)
results = cursor.fetchall()

In [74]:
query = """
SELECT A.DATE, SUM(A.AMOUNTS_SUM) OVER(ORDER BY A.DATE) AS OVER_SUM
FROM (
    SELECT TO_CHAR(TRANS_DATE, 'YYYY-MM-DD') AS DATE, SUM(AMOUNTS) AS AMOUNTS_SUM 
    FROM LEDGER.TRANSACTION
    WHERE TRANS_DATE >= DATE_TRUNC('MONTH', CURRENT_DATE - INTERVAL '1' MONTH)
    AND TRANS_DATE < DATE_TRUNC('MONTH', CURRENT_DATE)
    AND TYPE = 2
    GROUP BY TRANS_DATE
    ORDER BY TRANS_DATE ASC
) AS A
;
"""
cursor.execute(query)
results = cursor.fetchall()

In [75]:
payments_per_daily =  pd.DataFrame(results, columns = ['date', 'amounts'])
payments_per_daily

,date,amounts
0,2023-01-01,8100
1,2023-01-02,104600
2,2023-01-03,118600
3,2023-01-04,201050
4,2023-01-05,225200
5,2023-01-06,242400
6,2023-01-07,296700
7,2023-01-08,353550
8,2023-01-09,487750
9,2023-01-10,565350


In [76]:
query = """
SELECT A.DATE, SUM(A.AMOUNTS_SUM) OVER(ORDER BY A.DATE) AS OVER_SUM 
FROM (SELECT TO_CHAR(TRANS_DATE, 'YYYY-MM-DD') AS DATE, SUM(AMOUNTS) AS AMOUNTS_SUM
    FROM LEDGER.TRANSACTION
    WHERE TRANS_DATE >= DATE_TRUNC('MONTH', CURRENT_DATE)
    AND TYPE = 2
    GROUP BY TRANS_DATE
    ORDER BY TRANS_DATE ASC
) AS A
;
"""
cursor.execute(query)
results = cursor.fetchall()


In [77]:
payments_per_daily = pd.merge(payments_per_daily, pd.DataFrame(results, columns = ['date2', 'amounts2']), left_index=True, right_index = True, how = 'left')
payments_per_daily

,date,amounts,date2,amounts2
0,2023-01-01,8100,2023-02-01,19400
1,2023-01-02,104600,2023-02-02,32100
2,2023-01-03,118600,2023-02-03,119050
3,2023-01-04,201050,2023-02-04,258650
4,2023-01-05,225200,2023-02-05,336250
5,2023-01-06,242400,2023-02-06,352350
6,2023-01-07,296700,2023-02-07,369950
7,2023-01-08,353550,2023-02-08,381850
8,2023-01-09,487750,2023-02-09,394550
9,2023-01-10,565350,2023-02-10,401250


In [83]:
fig = go.Figure()
fig.add_trace(go.Scatter(y = payments_per_daily['amounts'], mode = 'lines', name = 'previous'))
fig.add_trace(go.Scatter(y = payments_per_daily['amounts2'], mode = 'lines', name = 'current'))
fig.update_layout(title = 'Compare Consumption b/w previous and current months')
fig.show()